## Loading packages

In [ ]:
# Import required packages
import warnings
import matplotlib.pyplot as plt
import itertools
import math

import cupid_utils.generic_fxns as gfxns

# Suppress Cartopy warnings (safe to keep even if Cartopy is not used)
warnings.filterwarnings("ignore")

In [ ]:
CESM_output_dir = ""
case_name = ""
case_nickname = ""
base_case_output_dir = None
base_case_name = None
base_case_nickname = None
vars = []

In [ ]:
# Want base case dir to default to control case value
if base_case_name is not None:
    if base_case_output_dir is None:
        base_case_output_dir = CESM_output_dir

### Read CESM output

In [ ]:
# Define cases with short labels

cases = {
    case_name: {
        "label": case_nickname,
        "offset": 0,  # TODO: make configurable
        "descr": "case",
        "path": CESM_output_dir,
    },
    base_case_name: {
        "label": base_case_nickname,
        "offset": 0,
        "descr": "baseline case",
        "path": base_case_output_dir,
    },
}

In [ ]:
variable_data = {}
for var in vars:
    if vars[var] == "Lab Sea":
        # define Lab sea region or other regions here
        region = [53.5, 65.3, 360 - 63.5, 360 - 47.5]

    # Compute global annual mean for each case
    regional_var = {}
    for casename, info in cases.items():
        label = info["label"]
        if vars[var] != "Global":
            regional_var[label] = gfxns.compute_var_lat_lon_ann(
                info["path"], casename, var, region[0], region[1], region[2], region[3]
            )
        elif vars[var] == "Global":
            if var != "RESTOM":
                regional_var[label] = gfxns.compute_var_g_ann(
                    info["path"], casename, var
                )
            elif var == "RESTOM":
                regional_var[label] = (
                    gfxns.compute_var_g_ann(info["path"], casename, var)
                    .rolling(year=5, center=True)
                    .mean()
                )
        variable_data[var] = regional_var

# 5 year running averaging but RESTOM in  legend is not running average

In [ ]:
# Set up plot
fig = plt.figure(figsize=(20, 10))
plt.subplots_adjust(hspace=0.3)
font_size_axis = 12

# Define automatic styles
colors = ["black", "red", "blue", "green", "orange"]
colors = ["black", "red", "blue", "green"]
colors = ["black", "red", "blue"]
linestyles = ["-", "--", ":", "-."]

total_number_vars = len(vars)

var_list = []
var_number = 0
for var in vars:
    var_list.append(var)
    var_number += 1
    regional_var = variable_data[var]
    # First subplot
    ax = fig.add_subplot(2, math.ceil(total_number_vars / 2), var_number)
    style_cycle = itertools.cycle(
        [(color, ls) for ls in linestyles for color in colors]
    )
    for casename, info in cases.items():
        label = info["label"]
        offset = info["offset"]
        regional_var_info = regional_var.get(label)
        if regional_var_info is not None:
            color, linestyle = next(style_cycle)
            ax.plot(
                regional_var_info.year + offset,
                regional_var_info,
                color=color,
                linestyle=linestyle,
                label=label,
                linewidth=2.0,
            )

    ax.set_xlabel("UNITS", fontsize=font_size_axis)  # TODO: make configurable!!
    # ax.set_ylabel("RESTOM (W/m2)", fontsize=font_size_axis) # Or TS(K)
    # ax.set_title("RESTOM (5-year running avg)", fontsize=20)
    ax.set_ylabel(var, fontsize=font_size_axis)
    ax.set_title(f"{var} ({vars[var]})", fontsize=20)
    ax.tick_params(axis="both", which="major", labelsize=14)
    # ax.set_ylim(0, 0.5)
    ax.grid()


# Adding legend with title
legend = ax.legend(
    loc="center left", bbox_to_anchor=(1.3, 0.5), fontsize="16", title=var_list
)
legend.get_title().set_fontsize("18")  # Setting the font size of the legend title

ax.grid()

plt.show()